In [1]:
"""
4장. word2vec 속도 개선
"""

'\n4장. word2vec 속도 개선\n'

In [3]:
import sys
sys.path.append('..')
from common import config
import numpy as np
from negative_sampling_layer import UnigramSampler
# GPU에서 실행하려면 아래 주석을 해제하세요(CuPy 필요).
# ===============================================
# config.GPU = True
# ===============================================
import pickle
from common.trainer import Trainer
from common.optimizer import Adam
from cbow import CBOW
from common.util import create_contexts_target, to_cpu, to_gpu
import ptb
from common.util import most_similar, analogy

In [2]:
# 4.1.2 Embedding 계층 구현
W = np.arange(21).reshape(7, 3)

In [3]:
W

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14],
       [15, 16, 17],
       [18, 19, 20]])

In [4]:
W[2]

array([6, 7, 8])

In [5]:
W[4]

array([12, 13, 14])

In [6]:
idx = np.array([1, 0, 3, 0])
W[idx]

array([[ 3,  4,  5],
       [ 0,  1,  2],
       [ 9, 10, 11],
       [ 0,  1,  2]])

In [12]:
class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None
        
    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out
    
    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0
        # dW[self.idx] = dout # 문제 발생
        np.add.at(dW, self.idx, dout)
        return None

In [13]:
# 4.2.4 다중 분류에서 이진 분류로(EmbeddingDot)
class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None
        
    # hidden layer의 뉴런 h, 단어 id의 numpy 배열 idx
    def forward(self, h, idx): 
        target_W = self.embed.forward(idx) # 단어 임베딩 계산 
        out = np.sum(target_W * h, axis = 1) # 내적 계산
        
        self.cache = (h, target_W)
        return out
    
    def backward(self, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0], 1)
        
        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        
        return dh

In [18]:
# 4.2.6 네거티브 샘플링

# 0~9까지 숫자 하나를 무작위로 샘플링
np.random.choice(10)

6

In [19]:
np.random.choice(10)

2

In [20]:
# words에서 하나만 무작위로 샘플링
words = ['you', 'say', 'goodbye', 'I', 'hello', '.']
np.random.choice(words)

'you'

In [24]:
# 5개만 무작위로 샘플링(중복 있음)
np.random.choice(words, size=5)

array(['hello', '.', '.', 'you', 'hello'], dtype='<U7')

In [25]:
# 5개만 무작위로 샘플링(중복 없음)
np.random.choice(words, size=5, replace=False)

array(['hello', '.', 'say', 'you', 'I'], dtype='<U7')

In [28]:
# 확률분포에 따라 샘플링
p = [0.5, 0.1, 0.05, 0.2, 0.05, 0.1]
np.random.choice(words, p=p)

'you'

In [31]:
# 4.2.6 네거티브 샘플링의 샘플링 기법
p = [0.7, 0.29, 0.01]
new_p = np.power(p, 0.75) # 0.75 제곱
new_p /= np.sum(new_p) # 0.75를 제곱한 확률분포의 총합
print(new_p)

[0.64196878 0.33150408 0.02652714]


In [38]:
corpus = np.array([0, 1, 2, 3, 4, 1, 2, 3])
power = 0.75
sample_size = 2

sampler = UnigramSampler(corpus, power, sample_size)
target = np.array([1, 3, 0]) # target으로 지정한 단어는 positive, 그 외의 단어는 negative로 샘플링
negative_sample = sampler.get_negative_sample(target)
print(negative_sample)

[[2 0]
 [1 0]
 [3 4]]


In [2]:
# 4.3.2 CBOW 모델 학습 코드

# 하이퍼파라미터 설정
window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 10

# 데이터 읽기
corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)

contexts, target = create_contexts_target(corpus, window_size)
if config.GPU:
    contexts, target = to_gpu(contexts), to_gpu(target)

# 모델 등 생성
model = CBOW(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

# 학습 시작
trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

# 나중에 사용할 수 있도록 필요한 데이터 저장
word_vecs = model.word_vecs
if config.GPU:
    word_vecs = to_cpu(word_vecs)
params = {}
params['word_vecs'] = word_vecs.astype(np.float16)
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word
pkl_file = 'cbow_params.pkl'
with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1)

| 에폭 1 |  반복 1 / 9295 | 시간 0[s] | 손실 4.16
| 에폭 1 |  반복 21 / 9295 | 시간 1[s] | 손실 4.16
| 에폭 1 |  반복 41 / 9295 | 시간 1[s] | 손실 4.15
| 에폭 1 |  반복 61 / 9295 | 시간 2[s] | 손실 4.12
| 에폭 1 |  반복 81 / 9295 | 시간 3[s] | 손실 4.05
| 에폭 1 |  반복 101 / 9295 | 시간 4[s] | 손실 3.92
| 에폭 1 |  반복 121 / 9295 | 시간 5[s] | 손실 3.78
| 에폭 1 |  반복 141 / 9295 | 시간 6[s] | 손실 3.62
| 에폭 1 |  반복 161 / 9295 | 시간 7[s] | 손실 3.49
| 에폭 1 |  반복 181 / 9295 | 시간 8[s] | 손실 3.35
| 에폭 1 |  반복 201 / 9295 | 시간 9[s] | 손실 3.25
| 에폭 1 |  반복 221 / 9295 | 시간 10[s] | 손실 3.16
| 에폭 1 |  반복 241 / 9295 | 시간 11[s] | 손실 3.09
| 에폭 1 |  반복 261 / 9295 | 시간 11[s] | 손실 3.02
| 에폭 1 |  반복 281 / 9295 | 시간 12[s] | 손실 2.98
| 에폭 1 |  반복 301 / 9295 | 시간 13[s] | 손실 2.93
| 에폭 1 |  반복 321 / 9295 | 시간 14[s] | 손실 2.84
| 에폭 1 |  반복 341 / 9295 | 시간 15[s] | 손실 2.83
| 에폭 1 |  반복 361 / 9295 | 시간 16[s] | 손실 2.82
| 에폭 1 |  반복 381 / 9295 | 시간 17[s] | 손실 2.78
| 에폭 1 |  반복 401 / 9295 | 시간 18[s] | 손실 2.77
| 에폭 1 |  반복 421 / 9295 | 시간 19[s] | 손실 2.73
| 에폭 1 |  반복 441 / 9295 | 시

| 에폭 1 |  반복 3561 / 9295 | 시간 169[s] | 손실 2.43
| 에폭 1 |  반복 3581 / 9295 | 시간 170[s] | 손실 2.43
| 에폭 1 |  반복 3601 / 9295 | 시간 171[s] | 손실 2.40
| 에폭 1 |  반복 3621 / 9295 | 시간 172[s] | 손실 2.41
| 에폭 1 |  반복 3641 / 9295 | 시간 172[s] | 손실 2.41
| 에폭 1 |  반복 3661 / 9295 | 시간 173[s] | 손실 2.42
| 에폭 1 |  반복 3681 / 9295 | 시간 174[s] | 손실 2.43
| 에폭 1 |  반복 3701 / 9295 | 시간 175[s] | 손실 2.40
| 에폭 1 |  반복 3721 / 9295 | 시간 176[s] | 손실 2.42
| 에폭 1 |  반복 3741 / 9295 | 시간 177[s] | 손실 2.42
| 에폭 1 |  반복 3761 / 9295 | 시간 178[s] | 손실 2.40
| 에폭 1 |  반복 3781 / 9295 | 시간 179[s] | 손실 2.41
| 에폭 1 |  반복 3801 / 9295 | 시간 179[s] | 손실 2.40
| 에폭 1 |  반복 3821 / 9295 | 시간 180[s] | 손실 2.43
| 에폭 1 |  반복 3841 / 9295 | 시간 181[s] | 손실 2.41
| 에폭 1 |  반복 3861 / 9295 | 시간 182[s] | 손실 2.40
| 에폭 1 |  반복 3881 / 9295 | 시간 183[s] | 손실 2.37
| 에폭 1 |  반복 3901 / 9295 | 시간 184[s] | 손실 2.43
| 에폭 1 |  반복 3921 / 9295 | 시간 185[s] | 손실 2.42
| 에폭 1 |  반복 3941 / 9295 | 시간 185[s] | 손실 2.40
| 에폭 1 |  반복 3961 / 9295 | 시간 186[s] | 손실 2.36
| 에폭 1 |  반복 

KeyboardInterrupt: 

In [4]:
# 4.3.3 CBOW 모델 평가

pkl_file = 'cbow_params.pkl'

with open(pkl_file, 'rb') as f:
    params = pickle.load(f)
    word_vecs = params['word_vecs']
    word_to_id = params['word_to_id']
    id_to_word = params['id_to_word']

# 가장 비슷한(most similar) 단어 뽑기
querys = ['you', 'year', 'car', 'toyota']
for query in querys:
    most_similar(query, word_to_id, id_to_word, word_vecs, top=5)

# 유추(analogy) 작업
print('-'*50)
analogy('king', 'man', 'queen',  word_to_id, id_to_word, word_vecs)
analogy('take', 'took', 'go',  word_to_id, id_to_word, word_vecs)
analogy('car', 'cars', 'child',  word_to_id, id_to_word, word_vecs)
analogy('good', 'better', 'bad',  word_to_id, id_to_word, word_vecs)


[query] you
 we: 0.6103515625
 someone: 0.59130859375
 i: 0.55419921875
 something: 0.48974609375
 anyone: 0.47314453125

[query] year
 month: 0.71875
 week: 0.65234375
 spring: 0.62744140625
 summer: 0.6259765625
 decade: 0.603515625

[query] car
 luxury: 0.497314453125
 arabia: 0.47802734375
 auto: 0.47119140625
 disk-drive: 0.450927734375
 travel: 0.4091796875

[query] toyota
 ford: 0.55078125
 instrumentation: 0.509765625
 mazda: 0.49365234375
 bethlehem: 0.47509765625
 nissan: 0.474853515625
--------------------------------------------------

[analogy] king:man = queen:?
 woman: 5.16015625
 veto: 4.9296875
 ounce: 4.69140625
 earthquake: 4.6328125
 successor: 4.609375

[analogy] take:took = go:?
 went: 4.55078125
 points: 4.25
 began: 4.09375
 comes: 3.98046875
 oct.: 3.90625

[analogy] car:cars = child:?
 children: 5.21875
 average: 4.7265625
 yield: 4.20703125
 cattle: 4.1875
 priced: 4.1796875

[analogy] good:better = bad:?
 more: 6.6484375
 less: 6.0625
 rather: 5.21875
 slow